# Task02. 搭建开发环境并运行、理解时序插补工作流

## 1. 开发环境配置

### PyPOTS开发环境支持多种安装方式, 你可以自由选择从源码安装, 通过pip安装PyPI上的发布版本或者使用conda从conda-forge的发行版进行环境配置, 如果你熟悉docker的使用方式, 也可以通过docker来获取我们已经为你配置好的PyPOTS开发环境容器

### 从下方选择一种你熟悉的安装方式来为PyPOTS配置Python开发环境

💡 **补充说明**：不同的安装方式适用于不同的使用场景。

- 从源码安装适合希望获取最新特性或参与开发的用户。
- pip 安装适合大多数用户，获取的是官方发布的稳定版本。
- conda 安装适合在 Anaconda 环境中使用，依赖管理更方便。
- docker 安装适合希望一键部署环境的用户，尤其适合在服务器上部署运行。

In [ ]:
# 从源码安装
# 🚀 从源码安装 PyPOTS：推荐用于获取最新开发版（可能包含最新特性，但稳定性略低）
# 如果你希望参与开发或想尝试最新功能，可以使用这种方式安装
!pip install https://github.com/WenjieDu/PyPOTS/archive/main.zip

In [ ]:
# 从PyPI安装 
# ✅ 从PyPI安装 PyPOTS：推荐用于稳定版本的安装
# 这种方式适合大多数用户，安装的是在 PyPI 上发布的最新稳定版本
!pip install pypots

In [ ]:
# 从conda-forge安装 (‼️请确定你熟悉conda的操作并且确认你的电脑上安装了conda)
!conda install conda-forge::pypots

In [ ]:
# 运行配置好PyPOTS开发环境的docker容器 (‼️请确定你熟悉docker的使用并且确认你的电脑上安装了docker)
!docker run -it --name pypots wenjiedu/pypots

## 2. 时间序列插补工作流

## 生成一个随机的时间序列数据集

In [1]:
# 导入必要的库
from benchpots.datasets import preprocess_physionet2012  # 导入physionet2012数据集预处理函数

# 加载并预处理physionet2012数据集
# subset="set-a": 使用set-a子集
# pattern="point": 使用点缺失模式
# rate=0.1: 设置缺失率为10%
physionet2012_dataset = preprocess_physionet2012(
    subset="set-a", 
    pattern="point", 
    rate=0.1,
)

# 打印数据集的键，查看包含哪些数据
print(physionet2012_dataset.keys())

2025-05-09 15:29:55 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-09 15:29:55 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-09 15:29:55 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-09 15:29:55 [INFO]: Loaded successfully!
2025-05-09 15:30:03 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-09 15:30:03 [INFO]: 22772 values masked out in the val set as ground truth, take 9.92% of the original observed values
2025-05-09 15:30:03 [INFO]: 28895 values masked out in the test set as ground truth, take 10.00% of the original observed values
2025-05-09 15:30:03 [INFO]: Total sample number: 3997
2025-05-09 15:30:03 [IN

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


In [3]:
# 导入numpy库用于数值计算
import numpy as np

# 创建测试集的缺失值指示掩码
# 通过异或运算(^)比较原始数据和缺失数据的NaN位置，得到真实缺失值的位置
physionet2012_dataset["test_X_indicating_mask"] = np.isnan(physionet2012_dataset["test_X"]) ^ np.isnan(physionet2012_dataset["test_X_ori"])

# 将原始测试数据中的NaN值替换为0
physionet2012_dataset["test_X_ori"] = np.nan_to_num(physionet2012_dataset["test_X_ori"])

# 构建训练集字典，只包含输入特征X
train_set = {
    "X": physionet2012_dataset["train_X"],
}

# 构建验证集字典，包含输入特征X和原始数据X_ori
val_set = {
    "X": physionet2012_dataset["val_X"],
    "X_ori": physionet2012_dataset["val_X_ori"],
}

# 构建测试集字典，包含输入特征X和原始数据X_ori
test_set = {
    "X": physionet2012_dataset["test_X"],
    "X_ori": physionet2012_dataset["test_X_ori"],
}

In [4]:
# 获取physionet2012数据集中特征的数量
# n_features表示数据集中每个时间步包含的变量/特征数量
physionet2012_dataset['n_features']

37

In [6]:
# 导入SAITS模型，这是一个用于时间序列数据插补的深度学习模型
from pypots.imputation import SAITS

# 初始化SAITS模型，设置模型参数
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],      # 时间步长，从数据集中获取
    n_features=physionet2012_dataset['n_features'], # 特征数量，从数据集中获取
    n_layers=3,                                     # Transformer编码器层数
    d_model=64,                                     # 模型维度
    n_heads=4,                                      # 注意力头数
    d_k=16,                                         # 每个注意力头的键维度
    d_v=16,                                         # 每个注意力头的值维度
    d_ffn=128,                                      # 前馈神经网络隐藏层维度
    dropout=0.1,                                    # Dropout比率，用于防止过拟合
    epochs=10,                                      # 训练轮数
)

2025-05-09 15:30:22 [INFO]: No given device, using default device: cuda
2025-05-09 15:30:22 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-09 15:30:22 [INFO]: Using customized MAE as the training loss function.
2025-05-09 15:30:22 [INFO]: Using customized MSE as the validation metric function.
2025-05-09 15:30:22 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 218,294


In [7]:
# 使用训练集和验证集对SAITS模型进行训练
# train_set: 包含训练数据的字典，其中"X"键对应的值包含输入特征
# val_set: 包含验证数据的字典，其中"X"键对应的值包含输入特征，"X_ori"键对应的值包含原始数据
# 训练过程中会自动使用之前设置的参数，包括epochs=10等超参数
saits.fit(train_set, val_set)

2025-05-09 15:30:35 [INFO]: Epoch 001 - training loss (MAE): 1.1070, validation MSE: 1.2067
2025-05-09 15:30:39 [INFO]: Epoch 002 - training loss (MAE): 0.7917, validation MSE: 1.1190
2025-05-09 15:30:42 [INFO]: Epoch 003 - training loss (MAE): 0.6935, validation MSE: 1.0924
2025-05-09 15:30:45 [INFO]: Epoch 004 - training loss (MAE): 0.6501, validation MSE: 1.0746
2025-05-09 15:30:49 [INFO]: Epoch 005 - training loss (MAE): 0.6189, validation MSE: 1.0564
2025-05-09 15:30:52 [INFO]: Epoch 006 - training loss (MAE): 0.5973, validation MSE: 1.0546
2025-05-09 15:30:55 [INFO]: Epoch 007 - training loss (MAE): 0.5858, validation MSE: 1.0432
2025-05-09 15:30:59 [INFO]: Epoch 008 - training loss (MAE): 0.5705, validation MSE: 1.0403
2025-05-09 15:31:02 [INFO]: Epoch 009 - training loss (MAE): 0.5576, validation MSE: 1.0374
2025-05-09 15:31:06 [INFO]: Epoch 010 - training loss (MAE): 0.5527, validation MSE: 1.0315
2025-05-09 15:31:06 [INFO]: Finished training. The best model is from epoch#10.


In [8]:
# 使用训练好的SAITS模型对测试集进行预测，生成缺失值填充结果
# test_set: 包含测试数据的字典，其中"X"键对应的值包含输入特征
# 返回的test_set_imputation_results是一个字典，其中重要的键值对为：
# - "imputation": 填充后的完整时间序列数据，形状为(n_samples, n_steps, n_features)
test_set_imputation_results = saits.predict(test_set)

In [9]:
# 从pypots库中导入计算均方误差(MSE)的函数
from pypots.nn.functional import calc_mse

# 计算测试集上的均方误差(MSE)
# test_set_imputation_results["imputation"]: SAITS模型对测试集的插补结果
# physionet2012_dataset["test_X_ori"]: 测试集的原始完整数据
# physionet2012_dataset["test_X_indicating_mask"]: 测试集的缺失值指示掩码
test_MSE = calc_mse(
            test_set_imputation_results["imputation"],
            physionet2012_dataset["test_X_ori"],
            physionet2012_dataset["test_X_indicating_mask"],
)
# 打印SAITS模型在测试集上的MSE评估结果
print(f"SAITS test_MSE: {test_MSE}")

SAITS test_MSE: 0.14154181286514345


In [ ]:
# 导入必要的库
from typing import Optional  # 用于类型提示
import matplotlib.pyplot as plt  # 用于绘图
import numpy as np  # 用于数值计算
import pandas as pd  # 用于数据处理
from pypots.utils.logging import logger  # 用于日志记录

# ⚠️ TODO: 优化该画图函数
def plot_data(
    X: np.ndarray,  # 输入数据，包含缺失值的时间序列
    X_ori: np.ndarray,  # 原始完整数据
    X_imputed: np.ndarray,  # 模型填充后的数据
    sample_idx: Optional[int] = None,  # 要可视化的样本索引，如果为None则随机选择
    n_rows: int = 10,  # 子图的行数
    n_cols: int = 4,  # 子图的列数
    fig_size: Optional[list] = None,  # 图形大小，如果为None则使用默认值
):
    """
    可视化时间序列数据的函数，用于比较原始数据、缺失数据和填充后的数据
    
    参数:
        X: 包含缺失值的时间序列数据，形状为(n_samples, n_steps, n_features)
        X_ori: 原始完整数据，形状与X相同
        X_imputed: 模型填充后的数据，形状与X相同
        sample_idx: 要可视化的样本索引
        n_rows: 子图的行数
        n_cols: 子图的列数
        fig_size: 图形大小
    """
    # 获取数据形状
    vals_shape = X.shape
    assert len(vals_shape) == 3, "vals_obs应该是一个3D数组，形状为(n_samples, n_steps, n_features)"
    n_samples, n_steps, n_features = vals_shape

    # 如果未指定样本索引，随机选择一个
    if sample_idx is None:
        sample_idx = np.random.randint(low=0, high=n_samples)
        logger.warning(f"⚠️ 未指定样本索引，随机选择样本 {sample_idx} 进行可视化。")

    # 设置默认图形大小
    if fig_size is None:
        fig_size = [24, 36]

    # 计算要显示的特征数量
    n_k = n_rows * n_cols
    K = np.min([n_features, n_k])  # 取特征数量和子图数量的较小值
    L = n_steps  # 时间步长
    
    # 设置图形参数
    plt.rcParams["font.size"] = 16
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(fig_size[0], fig_size[1]))

    # 为每个特征创建子图
    for k in range(K):
        # 创建数据框用于绘图
        df = pd.DataFrame({"x": np.arange(0, L), "val": X_imputed[sample_idx, :, k]})  # 填充后的数据
        df1 = pd.DataFrame({"x": np.arange(0, L), "val": X[sample_idx, :, k]})  # 缺失数据
        df2 = pd.DataFrame({"x": np.arange(0, L), "val": X_ori[sample_idx, :, k]})  # 原始数据
        
        # 计算子图位置
        row = k // n_cols
        col = k % n_cols
        
        # 绘制三种数据
        axes[row][col].plot(df1.x, df1.val, color="r", marker="x", linestyle="None")  # 缺失数据用红色x标记
        axes[row][col].plot(df2.x, df2.val, color="b", marker="o", linestyle="None")  # 原始数据用蓝色o标记
        axes[row][col].plot(df.x, df.val, color="g", linestyle="solid")  # 填充数据用绿色实线
        
        # 设置坐标轴标签
        if col == 0:
            plt.setp(axes[row, 0], ylabel="value")  # 第一列添加y轴标签
        if row == -1:
            plt.setp(axes[-1, col], xlabel="time")  # 最后一行添加x轴标签

    logger.info("绘图完成。请调用matplotlib.pyplot.show()显示图形。")


# 调用绘图函数，可视化测试集数据
plot_data(
    test_set["X"],  # 测试集输入数据
    test_set["X_ori"],  # 测试集原始数据
    test_set_imputation_results["imputation"],  # 模型填充结果
    5,  # 选择第5个样本
    n_rows=7,  # 7行子图
    n_cols=6,  # 6列子图
    fig_size=[100, 50]  # 图形大小
)